In [1]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


In [2]:
def answer_one():
    temp_df = df.sort_values('Gold', ascending=False) 
    return temp_df.index[0]

In [3]:
answer_one()

'United States'

In [8]:
def answer_two():
    df2 = df.copy()
    df2['medal_diff'] = (df2['Gold'] - df2['Gold.1']).abs()
    df2 = df2.sort_values('medal_diff', ascending=False)
    return df2.index[0]

In [9]:
answer_two()

'United States'

In [14]:
def answer_three():
    #filter golds
    df2 = df.copy()
    df2 = df2[(df2['Gold'] > 0) & (df2['Gold.1'] > 0)]
    df2['medal_diff'] = (df2['Gold'] - df2['Gold.1']) / df2['Gold.2']
    df2 = df2.sort_values('medal_diff', ascending=False)
    return df2.index[0]


In [15]:
answer_three()

'Bulgaria'

In [20]:
def answer_four():
    Points = pd.Series((df['Gold.2'] * 3) + (df['Silver.2'] * 2) + (df['Bronze.2'] * 1))
    return Points

In [24]:
type(answer_four())

pandas.core.series.Series

In [2]:
census_df = pd.read_csv('census.csv')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [20]:
def answer_five():
    census_working = census_df[census_df['SUMLEV'] == 50].copy()
    #census_working = census_working[census_working['SUMLEV']]
    census_working = census_working.set_index(['STNAME'])
    census_summary = census_working[['CTYNAME']].groupby(census_working.index).count().sort_values('CTYNAME', ascending=False)
    return census_summary.index[0]

In [21]:
answer_five()

'Texas'

In [22]:
def answer_six():
    census_working = census_df[census_df['SUMLEV'] == 50].copy()
    census_working = census_working.set_index(['STNAME'])
    census_working['RANK'] = census_working['CENSUS2010POP'].groupby(census_working.index).rank(ascending=False)
    census_working = census_working[census_working['RANK'].isin([1, 2, 3])]
    state_pop = census_working['CENSUS2010POP'].groupby(census_working.index).sum()
    state_pop = state_pop.sort_values(ascending = False)
    return [state_pop.index[0], state_pop.index[1], state_pop.index[2]]

In [23]:
answer_six()

['California', 'Texas', 'Illinois']

In [44]:
census_working.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
STNAME,,,,,,,,,,,,,,,,,,,,,
Alabama,40,3,6,1,0,Alabama,4779736,4780127,4785161,4801108,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
Alabama,50,3,6,1,1,Autauga County,54571,54571,54660,55253,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
Alabama,50,3,6,1,3,Baldwin County,182265,182265,183193,186659,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
Alabama,50,3,6,1,5,Barbour County,27457,27457,27341,27226,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
Alabama,50,3,6,1,7,Bibb County,22915,22919,22861,22733,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [63]:
(one, two) = [1+1, 1+2
             ]

In [7]:
census_df.iloc[:,7]

0       4779736
1         54571
2        182265
3         27457
4         22915
5         57322
6         10914
7         20947
8        118572
9         34215
10        25989
11        43643
12        13859
13        25833
14        13932
15        14972
16        49948
17        54428
18        13228
19        11539
20        37765
21        13906
22        80406
23        50251
24        43820
25        71109
26        79303
27        38319
28       104430
29        17241
         ...   
3163     131887
3164     389891
3165      52410
3166      24496
3167     166994
3168      74749
3169     563626
3170      36299
3171      11668
3172      46133
3173      15885
3174      13833
3175       7083
3176      40123
3177      13249
3178       4812
3179       8569
3180      91738
3181      18106
3182      75450
3183       2484
3184      28205
3185       8667
3186      29116
3187      10247
3188      43806
3189      21294
3190      21118
3191       8533
3192       7208
Name: CENSUS2010POP, Len

In [65]:
two

3

In [24]:
def answer_seven():
    census_working = census_df[census_df['SUMLEV'] == 50].copy()
    census_working = census_working.set_index(['CTYNAME'])
    left = [7,8,9,10,11]
    right = [8,9,10,11,12]
    for i,j in zip(left, right):
        census_working['diff' + str(i)] = (census_working.iloc[:,i] - census_working.iloc[:,j]).abs()
    max_change = census_working[['diff7', 'diff8', 'diff9', 'diff10', 'diff11']].max(axis=1)
    max_change = max_change.sort_values(ascending = False)
    return max_change.index[0]

In [25]:
answer_seven()

'Harris County'

In [50]:
def answer_eight():
    census_working = census_df[census_df['SUMLEV'] == 50].copy()
    census_working = census_working[(census_working['REGION'].isin([1,2])) \
                                    & (census_working['CTYNAME'].str.slice(0,10) == 'Washington') \
                                    & (census_working['POPESTIMATE2015'] > census_working['POPESTIMATE2014'])]
    return census_working[['STNAME', 'CTYNAME'] ]

In [51]:
answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County


In [42]:
goulet=census_df[census_df['CTYNAME'].str.slice(0,2) == 'Was']

In [43]:
goulet


,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,diff7,diff8,diff9,diff10,diff11


In [45]:
census_df['CTYNAME'].str.slice(0,2)

0       Al
1       Au
2       Ba
3       Ba
4       Bi
5       Bl
6       Bu
7       Bu
8       Ca
9       Ch
10      Ch
11      Ch
12      Ch
13      Cl
14      Cl
15      Cl
16      Co
17      Co
18      Co
19      Co
20      Co
21      Cr
22      Cu
23      Da
24      Da
25      De
26      El
27      Es
28      Et
29      Fa
        ..
3163    Wa
3164    Wa
3165    Wa
3166    Wa
3167    Wi
3168    Wo
3169    Wy
3170    Al
3171    Bi
3172    Ca
3173    Ca
3174    Co
3175    Cr
3176    Fr
3177    Go
3178    Ho
3179    Jo
3180    La
3181    Li
3182    Na
3183    Ni
3184    Pa
3185    Pl
3186    Sh
3187    Su
3188    Sw
3189    Te
3190    Ui
3191    Wa
3192    We
Name: CTYNAME, Length: 3193, dtype: object